In [1]:
# Setup
import pandas as pd
import altair as alt
import requests
from io import StringIO


In [9]:

weather_url = "https://raw.githubusercontent.com/duba1910/msds/main/Vital%20Skills/5304%20Visualization/final%20project/Rainier_Weather.csv"
climbing_url = "https://raw.githubusercontent.com/duba1910/msds/main/Vital%20Skills/5304%20Visualization/final%20project/climbing_statistics.csv"


response_weather = requests.get(weather_url)
data_weather = response_weather.content.decode('utf-8')
weather_data = pd.read_csv(StringIO(data_weather))

response_climbing = requests.get(climbing_url)
data_climbing = response_climbing.content.decode('utf-8')
climbing_data = pd.read_csv(StringIO(data_climbing))

# Check the data
climbing_data.info()
weather_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4077 entries, 0 to 4076
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                4077 non-null   object 
 1   Route               4077 non-null   object 
 2   Attempted           4077 non-null   int64  
 3   Succeeded           4077 non-null   int64  
 4   Success Percentage  4077 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 159.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464 entries, 0 to 463
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   464 non-null    object 
 1   Battery Voltage AVG    464 non-null    float64
 2   Temperature AVG        464 non-null    float64
 3   Relative Humidity AVG  464 non-null    float64
 4   Wind Speed Daily AVG   464 non-null    float64
 5   Wind Direction

Preliminary exploration

In [10]:
joined_data = pd.merge(climbing_data, weather_data, how="left", on=["Date"])
joined_data.head()

,Date,Route,Attempted,Succeeded,Success Percentage,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG
0,11/27/2015,Disappointment Cleaver,2,0,0.0,13.643750,26.321667,19.715000,27.839583,68.004167,88.496250
1,11/21/2015,Disappointment Cleaver,3,0,0.0,13.749583,31.300000,21.690708,2.245833,117.549667,93.660417
2,10/15/2015,Disappointment Cleaver,2,0,0.0,13.461250,46.447917,27.211250,17.163625,259.121375,138.387000
3,10/13/2015,Little Tahoma,8,0,0.0,13.532083,40.979583,28.335708,19.591167,279.779167,176.382667
4,10/9/2015,Disappointment Cleaver,2,0,0.0,13.216250,38.260417,74.329167,65.138333,264.687500,27.791292


In [11]:
# Remove missing values
joined_data.isna().sum()
joined_data = joined_data.dropna()
joined_data = joined_data[joined_data['Success Percentage'] <= 1]  
joined_data.describe()

,Attempted,Succeeded,Success Percentage,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG
count,1889.000000,1889.000000,1889.000000,1889.000000,1889.000000,1889.000000,1889.000000,1889.000000,1889.000000
mean,5.364743,2.573319,0.450365,13.502626,40.710507,47.869871,10.488669,155.508006,261.448846
std,3.908629,3.354267,0.457832,0.070525,9.114892,22.157661,9.141712,91.095224,110.563545
min,1.000000,0.000000,0.000000,12.987917,6.834833,12.363500,0.000000,13.490000,0.000000
25%,2.000000,0.000000,0.000000,13.453333,33.376667,30.057083,4.410292,55.200917,220.184875
50%,3.000000,2.000000,0.416667,13.491250,40.049167,46.140417,7.274917,171.633375,304.605708
75%,9.000000,4.000000,1.000000,13.546250,48.628750,65.174167,14.657875,235.496667,351.973292
max,12.000000,12.000000,1.000000,13.794583,56.153750,100.000000,65.138333,280.383333,368.056083


In [12]:
joined_data.head()

,Date,Route,Attempted,Succeeded,Success Percentage,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG
0,11/27/2015,Disappointment Cleaver,2,0,0.0,13.643750,26.321667,19.715000,27.839583,68.004167,88.496250
1,11/21/2015,Disappointment Cleaver,3,0,0.0,13.749583,31.300000,21.690708,2.245833,117.549667,93.660417
2,10/15/2015,Disappointment Cleaver,2,0,0.0,13.461250,46.447917,27.211250,17.163625,259.121375,138.387000
3,10/13/2015,Little Tahoma,8,0,0.0,13.532083,40.979583,28.335708,19.591167,279.779167,176.382667
4,10/9/2015,Disappointment Cleaver,2,0,0.0,13.216250,38.260417,74.329167,65.138333,264.687500,27.791292


In [22]:
joined_data['Group_ID'] = joined_data['Date'].astype(str) + '_' + joined_data['Route'] + '_' + joined_data['Attempted'].astype(str) + '_' + joined_data['Succeeded'].astype(str)
joined_data['Group_ID'] = joined_data['Group_ID'].apply(hash)
joined_data[(joined_data['Route'] == 'Disappointment Cleaver') & (joined_data['Date'] == '10/3/2015')].head()

,Date,Route,Attempted,Succeeded,Success Percentage,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG,Group_ID
5,10/3/2015,Disappointment Cleaver,10,0,0.0,13.5775,31.822917,62.337083,13.125042,153.931667,196.375208,1272329397530305784
6,10/3/2015,Disappointment Cleaver,2,0,0.0,13.5775,31.822917,62.337083,13.125042,153.931667,196.375208,-4904025732463163127
